<a target="_blank" href="https://colab.research.google.com/github/yandex-research/tabm/blob/main/example.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# TabM

This notebook provides a usage example of the `tabm` package from the
[TabM](https://github.com/yandex-research/tabm) project.

In [49]:

import math
import random
from copy import deepcopy
from typing import Any, Literal, NamedTuple, Optional

import numpy as np
import rtdl_num_embeddings  # https://github.com/yandex-research/rtdl-num-embeddings
import scipy.special
import sklearn.datasets
import sklearn.metrics
import sklearn.model_selection
import sklearn.preprocessing
import tabm
import torch
import torch.nn as nn
import torch.optim
from torch import Tensor

import numpy as np
import pandas as pd
from tqdm import tqdm
import os
from sklearn.model_selection import KFold

import optuna
from optuna.samplers import TPESampler
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_rank
from optuna.visualization import plot_slice
from optuna.visualization import plot_timeline

from sklearn.metrics import mean_absolute_percentage_error

In [50]:
seed = 0
random.seed(seed)
np.random.seed(seed + 1)
torch.manual_seed(seed + 2)
pass

# Dataset (Shell)

In [51]:
df_train = pd.read_csv('./dataset/train.csv')
df_test = pd.read_csv('./dataset/test.csv')
df_submission = pd.read_csv('./dataset/sample_solution.csv')

df_best = pd.read_csv('./Submissions_v3/sub_tabm_hpo_better_only (cur_best).csv')

feature_cols = df_train.columns[:55]
df_test_pred = pd.concat([df_test, df_best], axis=1)

In [52]:
df_test_pred = df_test_pred.drop(columns=['ID'])

In [ ]:

def apply_tabm_cv(trial, df_train, df_test_pred, feature_cols, target_col):
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    col_name = f'BlendProperty{target_col - 55 + 1}'
    df_oof_preds = df_train[[col_name]].copy()
    print(col_name)
    for fold, (train_idx, val_idx) in enumerate(kf.split(df_train)):
        train_fold = df_train.iloc[train_idx]
        val_fold = df_train.iloc[val_idx]
        
        score, val_preds, test_preds = apply_tabm(trial, train_fold, val_fold, df_test_pred, feature_cols, target_col)
        df_oof_preds.loc[df_train.index[val_idx], col_name] = val_preds
    
        # results_dir = f'./TabM/CV_b{target_col - 55 + 1}'
        # os.makedirs(results_dir, exist_ok=True)
        # np.save(f'{results_dir}/test_pred_fold_{fold}.npy', test_preds)
    
    mape = mean_absolute_percentage_error(df_train[col_name], df_oof_preds[col_name])
    oof_score = 100 - 90 * mape / 2.72
    os.makedirs(f'./Ensemble_tabm/BlendProperty{target_col-54}/oof', exist_ok=True)
    np.save(f'./Ensemble_tabm/BlendProperty{target_col-54}/oof/oof_b{target_col-54}_trial_{trial.number}_{oof_score}.npy', df_oof_preds[col_name].values)
    
    return 100 - 90 * mape / 2.72
        

def apply_tabm(trial, df_train, df_val, df_test_pred, feature_cols, target_col):

    # target_col: [55, 64]
    
    task_type = 'regression'
    target_col_name = df_train.columns[target_col]
    
    X_train = df_train[feature_cols].values.astype(np.float32)
    Y_train = df_train[target_col_name].values.astype(np.float32)
    
    X_val = df_val[feature_cols].values.astype(np.float32)
    Y_val = df_val[target_col_name].values.astype(np.float32)
    
    X_test = df_test_pred[feature_cols].values.astype(np.float32)
    Y_test = df_test_pred[target_col_name].values.astype(np.float32)
    
    n_num_features = len(feature_cols)

    data_numpy = {
        'train': {'x_num': X_train, 'y': Y_train},
        'val': {'x_num': X_val, 'y': Y_val},
        'test': {'x_num': X_test, 'y': Y_test},
    }


    for part, part_data in data_numpy.items():
        for key, value in part_data.items():
            # print(f'{part:<5}    {key:<5}    {value.shape!r:<10}    {value.dtype}')
            del key, value
        del part, part_data
        


    # Data Processing



    # Feature preprocessing.
    # NOTE
    # The choice between preprocessing strategies depends on a task and a model.

    # Simple preprocessing strategy.
    # preprocessing = sklearn.preprocessing.StandardScaler().fit(
    #     data_numpy['train']['x_num']
    # )
    
    # Advanced preprocessing strategy.
    # The noise is added to improve the output of QuantileTransformer in some cases.
    x_num_train_numpy = data_numpy['train']['x_num']
    noise = (
        np.random.default_rng(0)
        .normal(0.0, 1e-5, x_num_train_numpy.shape)
        .astype(x_num_train_numpy.dtype)
    )
    preprocessing = sklearn.preprocessing.QuantileTransformer(
        n_quantiles=max(min(df_train.shape[0] // 30, 1000), 10),
        output_distribution='normal',
        subsample=10**9,
    ).fit(x_num_train_numpy + noise)
    del x_num_train_numpy

    # Apply the preprocessing.
    for part in data_numpy:
        data_numpy[part]['x_num'] = preprocessing.transform(data_numpy[part]['x_num'])


    # Label preprocessing.
    class RegressionLabelStats(NamedTuple):
        mean: float
        std: float


    Y_train = data_numpy['train']['y'].copy()
    if task_type == 'regression':
        # For regression tasks, it is highly recommended to standardize the training labels.
        regression_label_stats = RegressionLabelStats(
            Y_train.mean().item(), Y_train.std().item()
        )
        Y_train = (Y_train - regression_label_stats.mean) / regression_label_stats.std
    else:
        regression_label_stats = None
        
        

    # Pytorch settings



    # Device
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    # Convert data to tensors
    data = {
        part: {k: torch.as_tensor(v, device=device) for k, v in data_numpy[part].items()}
        for part in data_numpy
    }
    Y_train = torch.as_tensor(Y_train, device=device)
    if task_type == 'regression':
        for part in data:
            data[part]['y'] = data[part]['y'].float()
        Y_train = Y_train.float()

    # Automatic mixed precision (AMP)
    # torch.float16 is implemented for completeness,
    # but it was not tested in the project,
    # so torch.bfloat16 is used by default.
    amp_dtype = (
        torch.bfloat16
        if torch.cuda.is_available() and torch.cuda.is_bf16_supported()
        else torch.float16
        if torch.cuda.is_available()
        else None
    )
    # Changing False to True can speed up training
    # of large enough models on compatible hardware.
    amp_enabled = False and amp_dtype is not None
    grad_scaler = torch.cuda.amp.GradScaler() if amp_dtype is torch.float16 else None  # type: ignore

    # torch.compile
    compile_model = False

    # fmt: off
    print(f'Device:        {device.type.upper()}')
    # print(f'AMP:           {amp_enabled}{f" ({amp_dtype})"if amp_enabled else ""}')
    # print(f'torch.compile: {compile_model}')
    # fmt: on





    # Model and optimizer

    # The best performance is usually achieved with `num_embeddings`
    # from the `rtdl_num_embeddings` package. Typically, `PiecewiseLinearEmbeddings`
    # and `PeriodicEmbeddings` perform best.
    # d_block, n_block, lr, d_embedding, n_bins, weight_decay
    embedding_type = trial.suggest_categorical('embedding_type', ['PeriodicEmbeddings', 'PiecewiseLinearEmbeddings'])
    # embedding_type = 'PiecewiseLinearEmbeddings'
    # Periodic embeddings.
    num_embeddings_periodic = rtdl_num_embeddings.PeriodicEmbeddings(n_num_features, lite=False)


    n_bins = trial.suggest_int('n_bins', 2, 128) # prev 48
    d_embedding = trial.suggest_int('d_embedding', 8, 32, step=4) # prev 16

    # Piecewise-linear embeddings.
    num_embeddings_piecewise = rtdl_num_embeddings.PiecewiseLinearEmbeddings(
        rtdl_num_embeddings.compute_bins(data['train']['x_num'], n_bins=n_bins),
        d_embedding=d_embedding,
        activation=False,
        version='B',
    )
    
    if embedding_type == 'PeriodicEmbeddings':
        num_embeddings = num_embeddings_periodic
    else:
        num_embeddings = num_embeddings_piecewise
        
    #  d_block, n_block, lr, d_embedding, n_bins, weight_decay
    n_blocks = trial.suggest_int("n_blocks", 1, 4)
    d_block = trial.suggest_int("d_block", 64, 1024, step=16)
    arch_type = trial.suggest_categorical('arch_type', ['tabm', 'tabm-mini'])
    # arch_type = 'tabm'
    
    n_classes = None
    
    model = tabm.TabM.make(
        n_num_features=n_num_features,
        cat_cardinalities=[],
        d_out=1 if n_classes is None else n_classes,
        num_embeddings=num_embeddings,
        n_blocks=n_blocks,
        d_block=d_block,
        arch_type=arch_type,
    ).to(device)
    
    lr = trial.suggest_float("lr", 1e-4, 5e-3, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-4, 1e-1, log=True)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    gradient_clipping_norm: Optional[float] = 1.0

    if compile_model:
        # NOTE
        # `torch.compile(model, mode="reduce-overhead")` caused issues during training,
        # so the `mode` argument is not used.
        model = torch.compile(model)
        evaluation_mode = torch.no_grad
    else:
        evaluation_mode = torch.inference_mode
        



    # A quick reminder: TabM represents an ensemble of k MLPs.
    #
    # The option below determines if the MLPs are trained
    # on the same batches (share_training_batches=True) or
    # on different batches. Technically, this option determines:
    # - How the loss function is implemented.
    # - How the training batches are constructed.
    #
    # `True` is recommended by default because of better training efficiency.
    # On some tasks, `False` may provide better performance.
    
    
    share_training_batches_var = trial.suggest_categorical('share_training_batches', ['T', 'F'])
    # share_training_batches_var = 'T'
    if share_training_batches_var == 'T':
        share_training_batches = True
    else:
        share_training_batches = False


    task_is_regression = task_type == 'regression'

    @torch.autocast(device.type, enabled=amp_enabled, dtype=amp_dtype)  # type: ignore[code]
    def apply_model(part: str, idx: Tensor) -> Tensor:
        return (
            model(
                data[part]['x_num'][idx],
                data[part]['x_cat'][idx] if 'x_cat' in data[part] else None,
            )
            .squeeze(-1)  # Remove the last dimension for regression tasks.
            .float()
        )


    base_loss_fn = (
        nn.functional.mse_loss if task_is_regression else nn.functional.cross_entropy
    )


    def loss_fn(y_pred: Tensor, y_true: Tensor) -> Tensor:
        # TabM produces k predictions. Each of them must be trained separately.

        # Regression:     (batch_size, k)            -> (batch_size * k,)
        # Classification: (batch_size, k, n_classes) -> (batch_size * k, n_classes)
        y_pred = y_pred.flatten(0, 1)

        if share_training_batches:
            # (batch_size,) -> (batch_size * k,)
            y_true = y_true.repeat_interleave(model.backbone.k)
        else:
            # (batch_size, k) -> (batch_size * k,)
            y_true = y_true.flatten(0, 1)

        return base_loss_fn(y_pred, y_true)


    @evaluation_mode()
    def evaluate(part: str) -> float:
        model.eval()

        # When using torch.compile, you may need to reduce the evaluation batch size.
        eval_batch_size = 32
        y_pred: np.ndarray = (
            torch.cat(
                [
                    apply_model(part, idx)
                    for idx in torch.arange(len(data[part]['y']), device=device).split(
                        eval_batch_size
                    )
                ]
            )
            .cpu()
            .numpy()
        )
        if task_type == 'regression':
            # Transform the predictions back to the original label space.
            assert regression_label_stats is not None
            y_pred = y_pred * regression_label_stats.std + regression_label_stats.mean

        # Compute the mean of the k predictions.
        if not task_is_regression:
            # For classification, the mean must be computed in the probability space.
            y_pred = scipy.special.softmax(y_pred, axis=-1)
        y_pred = y_pred.mean(1)

        y_true = data[part]['y'].cpu().numpy()
        score = (
            (sklearn.metrics.mean_absolute_percentage_error(y_true, y_pred))
            if task_type == 'regression'
            else sklearn.metrics.accuracy_score(y_true, y_pred.argmax(1))
        )
        # score = (
        #     -(sklearn.metrics.mean_squared_error(y_true, y_pred) ** 0.5)
        #     if task_type == 'regression'
        #     else sklearn.metrics.accuracy_score(y_true, y_pred.argmax(1))
        # )
        return float(100 - 90 * score / 2.72), y_pred  # The higher -- the better.


    # print(f'Test score before training: {evaluate("test")[0]:.4f}')








    n_epochs = 1_000_000_000
    train_size = df_train.shape[0]
    batch_size = 32
    epoch_size = math.ceil(train_size / batch_size)

    epoch = -1
    metrics = {'val': -math.inf, 'test': -math.inf}


    def make_checkpoint() -> dict[str, Any]:
        return deepcopy(
            {
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'epoch': epoch,
                'metrics': metrics,
            }
        )


    best_checkpoint = make_checkpoint()

    # Early stopping: the training stops if the validation score
    # does not improve for more than `patience` consecutive epochs.
    patience = 100
    remaining_patience = patience

    for epoch in range(n_epochs):
        batches = (
            # Create one standard batch sequence.
            torch.randperm(train_size, device=device).split(batch_size)
            if share_training_batches
            # Create k independent batch sequences.
            else (
                torch.rand((train_size, model.backbone.k), device=device)
                .argsort(dim=0)
                .split(batch_size, dim=0)
            )
        )
        for batch_idx in batches:
            model.train()
            optimizer.zero_grad()
            loss = loss_fn(apply_model('train', batch_idx), Y_train[batch_idx])
            if gradient_clipping_norm is not None:
                if grad_scaler is not None:
                    grad_scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad.clip_grad_norm_(
                    model.parameters(), gradient_clipping_norm
                )
            if grad_scaler is None:
                loss.backward()
                optimizer.step()
            else:
                grad_scaler.scale(loss).backward()  # type: ignore
                grad_scaler.step(optimizer)
                grad_scaler.update()

        metrics = {part: evaluate(part)[0] for part in ['val', 'test']}
        val_score_improved = metrics['val'] > best_checkpoint['metrics']['val']

        # print(
        #     f'{"*" if val_score_improved else " "}'
        #     f' [epoch] {epoch:<3}'
        #     f' [val] {metrics["val"]:.3f}'
        #     f' [test] {metrics["test"]:.3f}'
        # )

        if val_score_improved:
            best_checkpoint = make_checkpoint()
            remaining_patience = patience
        else:
            remaining_patience -= 1

        if remaining_patience < 0:
            break

    # To make final predictions, load the best checkpoint.
    model.load_state_dict(best_checkpoint['model'])

    val_score = best_checkpoint["metrics"]["val"]
    
    print('\n[Summary]')
    print(f'best epoch:  {best_checkpoint["epoch"]}')
    print(f'val score:  {best_checkpoint["metrics"]["val"]}')
    # print(f'test score: {best_checkpoint["metrics"]["test"]}')

    # df_submit = pd.read_csv('./from_BRACU_HPC/submission_autogluon_time_fraction_experimental_quality.csv')
    val_preds = evaluate('val')[1]
    test_preds = evaluate('test')[1]
    # df_submit[f'BlendProperty{target_col - 55 + 1}'] = test_preds
    # df_submit.to_csv(f'submission_tabm_test{target_col - 55 + 1}.csv', index=False)
    
    
    # if val_score >= 94:
    os.makedirs(f'./Ensemble_tabm/BlendProperty{target_col-54}/Test', exist_ok=True)
    os.makedirs(f'./Ensemble_tabm/BlendProperty{target_col-54}/Validation', exist_ok=True)
    np.save(f'./Ensemble_tabm/BlendProperty{target_col-54}/Test/test_b{target_col-54}_trial_{trial.number}_{val_score}.npy', test_preds)
    np.save(f'./Ensemble_tabm/BlendProperty{target_col-54}/Validation/val_b{target_col-54}_trial_{trial.number}_{val_score}.npy', val_preds)
        
    
    trial.report(val_score, epoch)

    if trial.should_prune():
        raise optuna.exceptions.TrialPruned()
    
    return best_checkpoint["metrics"]["val"], val_preds, test_preds

In [54]:
df_best_hparams = None

In [55]:
studies = []

In [ ]:
for target_col in tqdm([64, 59, 55, 56, 57, 58, 60, 62]):
    def objective(trial):
        return apply_tabm_cv(trial, df_train, df_test_pred, feature_cols, target_col)

    study = optuna.create_study(sampler=TPESampler(), direction='maximize')
    study.optimize(objective, n_trials=100)
    
    studies.append(study)
    map_hparams = study.best_params
    map_hparams['Target'] = f'BlendProperty{target_col - 55 + 1}'
    map_hparams['Score'] = study.best_value
    map_hparams['Best trial'] = study.best_trial.number
    df_cur_best = pd.DataFrame([map_hparams])
    df_best_hparams = pd.concat([df_best_hparams, df_cur_best])
    os.makedirs('./optuna/tabm_cv', exist_ok=True)
    df_best_hparams.to_csv('./optuna/tabm_cv/hparams_cv_v2.csv')

  0%|          | 0/8 [00:00<?, ?it/s][I 2025-07-22 21:15:42,309] A new study created in memory with name: no-name-41db4290-18db-401f-8ba9-2c031e4329f5


BlendProperty10
Device:        CUDA

[Summary]
best epoch:  164
val score:  93.18954441696405
Device:        CUDA

[Summary]
best epoch:  207
val score:  93.39510122423663
Device:        CUDA

[Summary]
best epoch:  84
val score:  94.26702475723098
Device:        CUDA

[Summary]
best epoch:  212
val score:  95.55612793740104
Device:        CUDA


[I 2025-07-22 21:16:41,309] Trial 0 finished with value: 93.50156278400769 and parameters: {'embedding_type': 'PiecewiseLinearEmbeddings', 'n_bins': 89, 'd_embedding': 20, 'n_blocks': 1, 'd_block': 496, 'arch_type': 'tabm-mini', 'lr': 0.00025732399569755945, 'weight_decay': 0.08368612754272077, 'share_training_batches': 'F'}. Best is trial 0 with value: 93.50156278400769.



[Summary]
best epoch:  241
val score:  91.10001533785287
BlendProperty10
Device:        CUDA

[Summary]
best epoch:  40
val score:  93.33156442598386
Device:        CUDA

[Summary]
best epoch:  124
val score:  93.19858701132677
Device:        CUDA

[Summary]
best epoch:  114
val score:  96.49212692919023
Device:        CUDA

[Summary]
best epoch:  116
val score:  93.93537059864578
Device:        CUDA


[I 2025-07-22 21:17:54,027] Trial 1 finished with value: 93.94541752669384 and parameters: {'embedding_type': 'PeriodicEmbeddings', 'n_bins': 93, 'd_embedding': 24, 'n_blocks': 3, 'd_block': 816, 'arch_type': 'tabm', 'lr': 0.000679385129747829, 'weight_decay': 0.003268767874364721, 'share_training_batches': 'T'}. Best is trial 1 with value: 93.94541752669384.


...


[Summary]
best epoch:  297
val score:  96.12506194149746
BlendProperty5
Device:        CUDA

[Summary]
best epoch:  924
val score:  95.85683364421129
Device:        CUDA

[Summary]
best epoch:  307
val score:  95.42129269855864
Device:        CUDA

[Summary]
best epoch:  239
val score:  96.98140548870845
Device:        CUDA

[Summary]
best epoch:  269
val score:  96.22151472003145
Device:        CUDA


[I 2025-07-22 23:09:32,539] Trial 0 finished with value: 95.98491250881413 and parameters: {'embedding_type': 'PiecewiseLinearEmbeddings', 'n_bins': 83, 'd_embedding': 16, 'n_blocks': 2, 'd_block': 880, 'arch_type': 'tabm', 'lr': 0.00018212948384630837, 'weight_decay': 0.0013091179985123755, 'share_training_batches': 'F'}. Best is trial 0 with value: 95.98491250881413.



[Summary]
best epoch:  1013
val score:  95.44351558913203
BlendProperty5
Device:        CUDA

[Summary]
best epoch:  202
val score:  98.32901829744087
Device:        CUDA


[W 2025-07-22 23:10:08,432] Trial 1 failed with parameters: {'embedding_type': 'PeriodicEmbeddings', 'n_bins': 110, 'd_embedding': 20, 'n_blocks': 4, 'd_block': 432, 'arch_type': 'tabm-mini', 'lr': 0.001307624248577046, 'weight_decay': 0.0064662383046997784, 'share_training_batches': 'T'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/FA020/miniconda3/envs/shell_ai/lib/python3.13/site-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_420383/1364787983.py", line 3, in objective
    return apply_tabm_cv(trial, df_train, df_test_pred, feature_cols, target_col)
  File "/tmp/ipykernel_420383/3637537109.py", line 10, in apply_tabm_cv
    score, val_preds, test_preds = apply_tabm(trial, train_fold, val_fold, df_test_pred, feature_cols, target_col)
                                   ~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tm

KeyboardInterrupt: 

In [ ]:
study.best_value

In [ ]:
map_hparams = studies[0].best_params
map_hparams['Target'] = "blend"

map_hparams

In [ ]:
# df1 = pd.DataFrame([studies[0].best_params])
# df2 = pd.DataFrame([studies[0].best_params])

# pd.concat([df1, df2])

In [ ]:
# for i in range(1, 10 + 1):
#     df = pd.read_csv(f'./from_BRACU_HPC/submission_tabm_test{i}.csv')
#     df_submission[f'BlendProperty{i}'] = df[f'BlendProperty{i}']

# df_submission.to_csv(f'./from_BRACU_HPC/submission_tabm_test_all.csv', index=False)

In [ ]:
from plotly.io import show

In [ ]:
# d_block, n_block, lr, d_embedding, n_bins, weight_decay
study = studies[9]
# optuna.visualization.plot_optimization_history(study)
optuna.visualization.plot_param_importances(study)

In [ ]:
optuna.visualization.plot_contour(study, params=['lr', 'n_bins'])

In [ ]:
optuna.visualization.plot_contour(study, params=['lr', 'weight_decay'])